In [1]:
!python -m pip install html2text

In [2]:
!python -m pip install mistletoe

https://github.com/webrecorder/warcio

In [3]:
!pip install warcio

In [4]:
!python -m pip install cdx_toolkit

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import json
import sys
import os.path
import re
import logging
import pandas as pd
import requests
import warcio
import mistletoe
import cdx_toolkit
import zlib

from warcio import ArchiveIterator
from contextlib import closing
from html2text import HTML2Text
from bs4 import BeautifulSoup
from pathlib import Path
from IPython.display import HTML as HTML_raw, display

In [7]:
websites = pd.read_csv("./ccmain/ccmain_2021-04_Long2.txt", header = None, sep = '|')

In [8]:
websites.head()

,0,1,2,3
0,NaN,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
1,NaN,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
2,NaN,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
3,NaN,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...,NaN
4,NaN,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...,NaN


In [9]:
websites.columns

Int64Index([0, 1, 2, 3], dtype='int64')

In [10]:
websites.drop([0, 3], axis=1, inplace=True)
websites.head()

,1,2
0,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
1,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...
4,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...


In [11]:
websites.columns = ['url_link', 'warc_file']
websites.head()

,url_link,warc_file
0,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
1,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...
4,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...


In [12]:
websites.url_link.value_counts()

https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      7
https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe                                                               

In [13]:
websites.url_link.value_counts().keys()

Index(['https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ',
       'https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe                                                  

In [14]:
urlList = []
for i in websites.url_link.value_counts().keys():
    urlList.append(i.replace(' ',''))
    
print(urlList[:10])

['https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington', 'https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/152230/senior-software-engineer-w-strich-m-strich-d-sap-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.xn--itbaw-ova.de/jobs/151789/werkstudent-w-strich-m-strich-d-softwareentwicklung-java-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/152535/consultant-w-strich-m-strich-d-microsoft-business-intelligence-in-stuttgart-und-karlsruhe', 'https://help.workable.com/hc/en-us/articles/360053410833-Can-I-use-Google-Analytics-on-my-careers-page-', 'https://www.xn--itbaw-ova.de/jobs/152501/consultant-w-strich-m-strich-d-sap-abap-in-stuttgart-karlsruhe-und-reutlingen', 'https://help.workable.com/hc/en-us/articles/115012441868-Which-job-boards-can-I-post-to-through-Workable-', 'https://www.workit

In [15]:
statesList = ["USA","AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA",
              "MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX",
              "UT","VT","VA","WA","WV","WI","WY","DC","GU","MH","MP","PR","VI"]

In [18]:
fout = open("2021JAN_USAjobAD.txt","w")
for i in range(0, len(urlList)):
    filename = "./dat/"+str(i+1)+"_2021JAN_jobAD.txt"
    if (os.path.exists(filename)):
        with open(filename, encoding='utf-8') as fin:
            #all_USAs = re.findall('USA', fin.read())
            all_USAs = re.findall(r"(?=("+'|'.join(statesList)+r"))", fin.read())
            nUSAs = len(all_USAs)
            print(filename,' ',nUSAs)
            if nUSAs > 0:
                fout.write(str(i+1)+'_2021JAN_jobAD.txt'+' '+str(nUSAs)+'\n')
            else:
                continue
    else:
        continue
            
fout.close()

./dat/1_2021JAN_jobAD.txt   49
./dat/11_2021JAN_jobAD.txt   3
./dat/12_2021JAN_jobAD.txt   0
./dat/21_2021JAN_jobAD.txt   4
./dat/22_2021JAN_jobAD.txt   3
./dat/23_2021JAN_jobAD.txt   3
./dat/27_2021JAN_jobAD.txt   4
./dat/28_2021JAN_jobAD.txt   4
./dat/33_2021JAN_jobAD.txt   0
./dat/34_2021JAN_jobAD.txt   0
./dat/39_2021JAN_jobAD.txt   4
./dat/41_2021JAN_jobAD.txt   4
./dat/42_2021JAN_jobAD.txt   0
./dat/45_2021JAN_jobAD.txt   3
./dat/49_2021JAN_jobAD.txt   1
./dat/53_2021JAN_jobAD.txt   24
./dat/54_2021JAN_jobAD.txt   45
./dat/55_2021JAN_jobAD.txt   17
./dat/56_2021JAN_jobAD.txt   8
./dat/62_2021JAN_jobAD.txt   15
./dat/65_2021JAN_jobAD.txt   3
./dat/69_2021JAN_jobAD.txt   0
./dat/74_2021JAN_jobAD.txt   3
./dat/84_2021JAN_jobAD.txt   6
./dat/87_2021JAN_jobAD.txt   17
./dat/91_2021JAN_jobAD.txt   4
./dat/92_2021JAN_jobAD.txt   0
./dat/96_2021JAN_jobAD.txt   5
./dat/108_2021JAN_jobAD.txt   3
./dat/115_2021JAN_jobAD.txt   1
./dat/116_2021JAN_jobAD.txt   1
./dat/117_2021JAN_jobAD.txt   1

./dat/836_2021JAN_jobAD.txt   177
./dat/837_2021JAN_jobAD.txt   2
./dat/841_2021JAN_jobAD.txt   3
./dat/844_2021JAN_jobAD.txt   10
./dat/847_2021JAN_jobAD.txt   1
./dat/848_2021JAN_jobAD.txt   0
./dat/850_2021JAN_jobAD.txt   14
./dat/851_2021JAN_jobAD.txt   0
./dat/853_2021JAN_jobAD.txt   9
./dat/855_2021JAN_jobAD.txt   11
./dat/859_2021JAN_jobAD.txt   8
./dat/860_2021JAN_jobAD.txt   53
./dat/864_2021JAN_jobAD.txt   177
./dat/873_2021JAN_jobAD.txt   1
./dat/875_2021JAN_jobAD.txt   5
./dat/876_2021JAN_jobAD.txt   10
./dat/877_2021JAN_jobAD.txt   205
./dat/882_2021JAN_jobAD.txt   3
./dat/883_2021JAN_jobAD.txt   14
./dat/884_2021JAN_jobAD.txt   4
./dat/886_2021JAN_jobAD.txt   0
./dat/888_2021JAN_jobAD.txt   11
./dat/891_2021JAN_jobAD.txt   0
./dat/892_2021JAN_jobAD.txt   10
./dat/894_2021JAN_jobAD.txt   3
./dat/896_2021JAN_jobAD.txt   14
./dat/897_2021JAN_jobAD.txt   0
./dat/898_2021JAN_jobAD.txt   7
./dat/902_2021JAN_jobAD.txt   0
./dat/904_2021JAN_jobAD.txt   1
./dat/906_2021JAN_jobAD.t